In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 250
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()
PATH = 'soft-os-wf'
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc_v1 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)

if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
from ag_ep import *
train(models, EUBO_init_eta_wf, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (148s),  symKL_DB_eta: 197.056,  symKL_DB_z: 215.668,  loss: -991.731,  ess: 3.608,  kl_eta_ex: 506.641,  kl_eta_in: 138.410,  kl_z_ex: 137.853,  kl_z_in: 24.121
epoch: 1\250 (147s),  symKL_DB_eta: 71.381,  symKL_DB_z: 51.237,  loss: -688.736,  ess: 4.623,  kl_eta_ex: 147.683,  kl_eta_in: 165.857,  kl_z_ex: 26.926,  kl_z_in: 9.159
epoch: 2\250 (147s),  symKL_DB_eta: 56.583,  symKL_DB_z: 36.728,  loss: -635.521,  ess: 4.903,  kl_eta_ex: 114.646,  kl_eta_in: 193.201,  kl_z_ex: 18.764,  kl_z_in: 6.452
epoch: 3\250 (146s),  symKL_DB_eta: 41.789,  symKL_DB_z: 21.026,  loss: -562.313,  ess: 5.271,  kl_eta_ex: 78.102,  kl_eta_in: 120.042,  kl_z_ex: 10.362,  kl_z_in: 3.978
epoch: 4\250 (147s),  symKL_DB_eta: 22.696,  symKL_DB_z: 11.780,  loss: -475.886,  ess: 5.695,  kl_eta_ex: 33.078,  kl_eta_in: 43.069,  kl_z_ex: 5.317,  kl_z_in: 2.011
epoch: 5\250 (148s),  symKL_DB_eta: 15.658,  symKL_DB_z: 8.016,  loss: -451.439,  ess: 5.997,  kl_eta_ex: 18.652,  kl_eta_in: 24.294,  kl_z_ex: 3

In [8]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)